# センサモデル

In [2]:
import numpy as np
import quaternion

c:\users\zzmasubuj\miniconda\envs\masu_001\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)


In [3]:
import sys
sys.path.append('../')
sys.path.append('../utility')

In [4]:
#座標系クラスのインポート
from COORDINATION_SYSTEMS import ECEF, LLA, NED, FRD, AC_ATTITUDE, LOCAL, SENSOR_ATTITUDE, FOCAL_POSITION, FOCAL_ANGLE
#WGS84座標変換クラスのインポート
from WGS84_COORDINATE_CONVERT import WGS84_COORDINATION_CONVERT
#座標変換クラスのインポート
from ATTITUDE_CONVERT import ATTITUDE_CONVERT
#センサ取得情報クラスのインポート
from SENSOR_FOCAL import SENSOR_FOCAL
#センサ性能クラスのインポート
from SENSOR_PERFORMANCE import FOV

クラス定義

In [5]:
class SENSOR_RECOGNITION():
    def __init__ (self):
        self.detected = 0
        self.oriented = 0
        self.recognized = 0
        self.identified = 0
        
    def get_recognition_level(self, own_lla =LLA(), own_att =AC_ATTITUDE(), sens_att = SENSOR_ATTITUDE(), opp_lla=LLA(), fov = FOV(),sensor_range=0):
        #用途：センサの認知判定を行う。
        #入力：自機位置 緯度経度高度　単位：°　（オイラーシーケンス：経度→緯度）
        #入力：自機姿勢 ロール、ピッチ、ヨー 単位：°（オイラーシーケンス：ヨー→ピッチ→ロール）
        #入力：センサ視軸 スピン、EL、AZ 単位：° （オイラーシーケンス：投影AZ→投影EL）
        #入力：相手機位置 緯度経度高度 単位：°（オイラーシーケンス：経度→緯度）
        #入力：センサ覆域（画像なら視野角） 覆域AZ、覆域EL 単位：°（オイラーシーケンス：覆域AZ→覆域EL）
        #出力：判定レベル（detection,orientation,recognition,identification) 単位：0=false 1=true
        
        #self.own_lla = own_lla
        #self.own_att = own_att
        #self.sens_att = sens_att
        #self.opp_lla = opp_lla
        
        
        #距離判定
        distance = SENSOR_FOCAL().get_focal_distance(own_lla, opp_lla)
        
        
        #視野判定
        focal = SENSOR_FOCAL().get_focal_angle(own_lla, own_att, sens_att, opp_lla)
        
        #recog_level = self.RECOGNITION_LEVEL(detected = 3, oriented = 3, recognized = 3, identified = 3)

        self.detected = 0
        self.oriented = 0
        self.recognized = 0
        self.identified = 0
        
        if(distance < sensor_range):
            if(focal.focal_az < fov.az):
                if(focal.focal_el < fov.el):      
                    self.detected = 1
                    self.oriented = 1
                    self.recognized = 1
                    self.identified = 1
        
        return  self.detected, self.oriented, self.recognized, self.identified

## 使い方

In [7]:
own_lla = LLA(lat = 36.0, lon =135.1, alt=10000) #自機位置
own_att = AC_ATTITUDE(roll =0, pitch = 0,yaw =0) #自機姿勢
sens_att =SENSOR_ATTITUDE(spin =0, el = 80, az =0) #センサ視軸
opp_lla = LLA(lat =36, lon = 135, alt = 12000) #相手機位置
sensor_fov=FOV(nAz= 5,nEl =5,nSize=0) #センサ覆域
sensor_range = 10000 #センサレンジ　単位m

In [8]:
SENSOR_RECOGNITION().get_recognition_level(own_lla, own_att,sens_att,opp_lla,sensor_fov, sensor_range)

(1, 1, 1, 1)